# Step-by-Step Guide to Deploying an image Classification Model using Flask & Docker
In this tutorial, we'll embark on a comprehensive journey, exploring the foundational elements required to transition a Machine/Deep Learning project from a mere concept to a live production application—one that users can seamlessly engage with.

We will take a deep dive into a hands-on example: crafting a user-friendly web interface for image recognition, powered by the classical VGG19 pre-trained model.

Deploying a model into production is paramount. It's the bridge that transforms our sophisticated algorithms and intricate data science workflows into tangible, real-world applications, enhancing user experiences and solving practical problems.
In order to fulfill our objective we need to follow this 4 big steps:


## Creating/Importing the model
The Convolutional Neural Network (CNN) that I've chosen to leverage for our image classification task is VGG-19, a deep learning model heralded for its efficacy. Developed by the Visual Geometry Group at Oxford, VGG-19 is a part of the VGG family, renowned for its depth and the consistency of its architecture.

The choice of VGG-19 is rooted in its robustness and proven accuracy in discerning intricate patterns and features in images, a capability cultivated through its training on millions of images from the ImageNet database.

Within our `model.py` file:

- **Loading the Model**: We've set up the necessary code to initialize and load the pre-trained VGG-19 weights. This allows us to harness the power of this deep learning model without undergoing the extensive and time-consuming training process from scratch.

- **Image Preprocessing**: Images from diverse sources come in various sizes, resolutions, and formats. To ensure our VGG-19 model can interpret and analyze these images, they need to be transformed into a consistent format. Our preprocessing step handles tasks like resizing images to the requisite dimensions, normalizing pixel values, and potentially augmenting the data to enhance prediction accuracy.

- **Making Predictions**: The final section of the model.py is geared towards taking a preprocessed image, feeding it through our VGG-19 model, and obtaining the predicted class or label. The output is typically a probability distribution over all possible classes, and the class with the highest probability becomes our model's prediction.

In [2]:
#Setting up everything we need 
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model

from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
base_model = VGG19(weights=None, include_top=True)  # 'weights=None' ensures that we only get the architecture
base_model.load_weights('./vgg19_weights_tf_dim_ordering_tf_kernels.h5')


2023-09-21 01:08:08.511944: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-21 01:08:08.637192: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...



### process_image

**Purpose**: This function prepares an image for predictions using the VGG19 model. 

**Description**:
- **Input**: The function takes in an image as an argument.
- **Steps**:
    1. Converts the image pixels into a numpy array using the `img_to_array` function.
    2. Reshapes the image data to match the input shape expected by the VGG model. This essentially adds an extra dimension to the image, turning it from a 3D array (height, width, channels) into a 4D array (batch size, height, width, channels), even if the batch size is just 1.
    3. Uses `preprocess_input` from Keras's VGG19 utilities to further preprocess the image. This function takes care of tasks like pixel normalization and other necessary transformations that make the image compatible with the VGG19 model.
- **Output**: Returns the processed image that's now ready for predictions with the VGG19 model.

---

### predict_class

**Purpose**: Given a processed image, this function predicts the class (or label) of the image using the VGG19 model.

**Description**:
- **Input**: The function takes in a processed image as an argument.
- **Steps**:
    1. Uses the `base_model` (which should be an instance of VGG19 loaded earlier) to predict the probability distribution across all classes for the given image.
    2. Decodes these probabilities into class labels using the `decode_predictions` utility from Keras's VGG19. This function translates the raw probabilities into human-readable class labels.
    3. Retrieves the topmost prediction, i.e., the label with the highest probability.
- **Output**: Returns two values:
    1. `prediction` - the predicted class or label for the image.
    2. `percentage` - the confidence (probability) of the prediction in percentage format.



In [3]:
def process_image(image):
    '''
    Make an image ready-to-use by VGG19
    '''
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    # prepare the image for the VGG model
    image = preprocess_input(image)

    return image

def predict_class(image):
    '''
    Predict and render the class of a given image 
    '''
    # predict the probability across all output classes
    yhat = base_model.predict(image)
    # convert the probabilities to class labels
    label = decode_predictions(yhat)
    # retrieve the most likely result, e.g. highest probability
    label = label[0][0]
    # return the classification
    prediction = label[1]
    percentage = '%.2f%%' % (label[2]*100)

    return prediction, percentage

In [4]:
image = load_img('./cute_cat.jpg', target_size=(224, 224))
image = process_image(image)
prediction, percentage = predict_class(image)
print(prediction, percentage)

35363/35363 [==============================] - 0s 1us/step
Egyptian_cat 73.31%


## Creating a FLASK App
Now we need to create a Flask App in order to interact with our model

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from keras.preprocessing.image import load_img
from model import process_image, predict_class
from werkzeug.utils import secure_filename
import os

app = Flask(__name__)

# Configuration
UPLOAD_FOLDER = './static/img'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/home', methods=['GET', 'POST'])
def home():
    welcome = "Hello, World !"
    return welcome

@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Check if the post request has the file part
        if 'photo' not in request.files:
            return redirect(request.url)
        file = request.files['photo']
        
        # If user does not select a file, the browser submits an empty file
        if file.filename == '':
            return redirect(request.url)
        
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)

            # Load, process, and predict
            image = load_img(filepath, target_size=(224, 224))
            image = process_image(image)
            prediction, percentage = predict_class(image)

            answer = "For {} : <br>The prediction is : {} <br>With probability = {}".format(filename, prediction, percentage)
            return answer

    return render_template('upload.html')


if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)



**Import Statements**:
```markdown
- `Flask`: The main class for our Flask web application.
- `render_template`: Used to render HTML templates.
- `request`: Helps in accessing incoming data.
- `redirect, url_for`: Utilities to handle redirections and URL generations.
- `load_img`: Utility from Keras to load images.
- `process_image, predict_class`: Our custom functions to preprocess an image and predict its class.
- `secure_filename`: Werkzeug utility to ensure the filename is safe to use.
- `os`: Standard library for OS-level operations.
```

**App Configuration**:
```markdown
- `UPLOAD_FOLDER`: Defines where the uploaded images will be stored.
- `ALLOWED_EXTENSIONS`: Set of allowed file extensions for image uploads.
- `app.config['UPLOAD_FOLDER']`: Sets the upload folder in the Flask app's configuration.
```

**Utility Function - `allowed_file`**:
```markdown
- A function that checks if the provided filename has a permitted extension.
```

**Home Route**:
```markdown
- `@app.route('/home', methods=['GET', 'POST'])`: Defines the home route.
- Simply returns a welcome message.
```

**Upload Route**:
```markdown
- `@app.route('/upload', methods=['GET', 'POST'])`: Defines the upload route which handles both GET and POST requests.
- `request.method == 'POST'`: Checks if the current request is a POST request.
- `if 'photo' not in request.files`: Ensures that a file was uploaded.
- `file.filename == ''`: Checks if the filename is empty (no file selected).
- `allowed_file(file.filename)`: Uses our utility function to check if the uploaded file has an allowed extension.
- `secure_filename(file.filename)`: Makes sure the filename is safe to use.
- `os.path.join(...)`: Constructs the full path where the file should be saved.
- `file.save(filepath)`: Saves the uploaded file to the specified path.
- `load_img(...)`: Loads the saved image with a specific target size.
- `process_image(image)`: Processes the image so it's ready for prediction.
- `predict_class(image)`: Predicts the class of the processed image.
```

**Main Execution**:
```markdown
- `if __name__ == '__main__'`: Ensures that the Flask app only runs if this script is executed as the main program.
- `app.run(host='0.0.0.0', debug=True)`: Starts the Flask app in debug mode and makes it accessible to any IP (not just localhost).
```



## Containerization with Docker

![Docker logo](Docker-Symbol.png)

In today's world of microservices and scalable applications, having a consistent environment is paramount to ensure smooth operations and to prevent the notorious "it works on my machine" syndrome. That's where Docker comes in!

### What is Docker?

Docker is a platform that uses containerization technology to "containerize" your applications, encapsulating them into packages called containers. These containers bundle an application's code, configurations, dependencies, and runtime into a single unit, ensuring it runs consistently across various environments.

### Advantages of Docker:

- **Consistency**: Since everything your application needs to run is bundled into a container, you eliminate the "works on my machine" problem.
  
- **Isolation**: Each container runs in isolation, ensuring that the environment remains pure and there are no conflicting dependencies.
  
- **Scalability and Portability**: Docker containers are lightweight, making it easier to scale microservices and move them between systems.

### Steps to Dockerize Our Flask Application:

1. **Installation**:
   
   If you haven't already, the first thing to do is to [download and install Docker](https://docs.docker.com/get-docker/).

2. **List Dependencies**:
   
   Ensure you have a `requirements.txt` file in your main directory. This file should list all the necessary Python packages required for your application. 

   Example:
   ```
   Flask==x.x.x
   tensorflow==x.x.x
   keras==x.x.x
   werkzeug==x.x.x
   ... (and so on)
   ```

3. **Dockerfile Creation**:
   
   The Dockerfile is essentially your recipe for how Docker should build the container. Create a file named `Dockerfile` (with no extension) in your directory. This file will contain instructions to set up the environment.

   Example:
   ```Dockerfile
   FROM python:3.10

   WORKDIR /app

   COPY requirements.txt requirements.txt
   RUN pip install -r requirements.txt

   COPY . .

   CMD ["python", "app.py"]
   ```

4. **Build the Docker Image**:

   In a terminal or command prompt, navigate to the directory containing your Dockerfile and run:
   
   ```bash
   docker build -t recog_container:api .
   ```

   This tells Docker to build an image using the Dockerfile in the current directory, tag it as `recog_container:api`, and build from the current context (`.`).

5. **Run the Docker Container**:

   With the image built, you can now run the container:

   ```bash
   docker run -p 5000:5000 -d recog_container:api
   ```

   This runs the container and maps port 5000 on your machine to port 5000 on the container. The `-d` flag tells Docker to run the container in the background.

6. **Access the Application**:

   Once the container is running, you should be able to access your Flask app by navigating to:

   [http://localhost:5000/upload](http://localhost:5000/upload)


## Go a step further: Deploying Dockerized Apps on Heroku

![Heroku Logo](https://www.herokucdn.com/deploy/button.svg)

### Introduction to Heroku

Heroku is a cloud platform that lets you build, deliver, monitor, and scale applications. The beauty of Heroku lies in its simplicity and integration capabilities. You can deploy code written in most programming languages, and it abstracts away infrastructure management, allowing developers to focus solely on their code.

### Why Heroku?

- **Ease of Use**: Heroku is user-friendly and makes the deployment process very streamlined.
  
- **Integrated Data Services**: It offers a range of database and caching options.
  
- **Scalability**: Easily scale your application vertically or horizontally.
  
- **Extensibility**: The Heroku Elements marketplace has numerous add-ons for databases, caching, monitoring, and more.

### Steps to Deploy Your Dockerized Flask Application on Heroku:

1. **Setup Heroku**:
   
   If you haven't already, sign up for a [Heroku account](https://signup.heroku.com/). Download and install the [Heroku CLI](https://devcenter.heroku.com/articles/heroku-cli).

2. **Login to Heroku CLI**:
   
   Open a terminal or command prompt and run:
   
   ```bash
   heroku login
   ```

   This will open a web page where you can log in. Once done, return to the terminal.

3. **Container Login**:
   
   Log in to the Heroku container service:

   ```bash
   heroku container:login
   ```

4. **Create a Heroku App**:

   ```bash
   heroku create your-app-name
   ```

   Replace `your-app-name` with a unique name for your app.

5. **Navigate to Your App's Directory**:
   
   Make sure you are in the directory where your `Dockerfile` resides.

6. **Push Your Docker Container**:

   ```bash
   heroku container:push web --app your-app-name
   ```

   This pushes the Docker container of your app to Heroku.

7. **Release the Application**:

   ```bash
   heroku container:release web --app your-app-name
   ```

   This will release your app, making it accessible via the web.

8. **Open & View Your App**:

   You can either navigate to your app's URL in the browser (which is usually in the format `https://your-app-name.herokuapp.com/`) or you can use the Heroku CLI:

   ```bash
   heroku open --app your-app-name
   ```

9. **Optional - Setting Up Config Vars**:

   If your app relies on environment variables, you can set them on Heroku through the dashboard or the CLI. For instance:

   ```bash
   heroku config:set MY_ENV_VAR=value --app your-app-name
   ```
